In [80]:
# Import the CARLA Python API library and some utils
import carla 
import math 
import random 
import time 
import pandas as pd

from send_to_carla import run

In [81]:
def setup_world():
    
    
    client = carla.Client('localhost', 2000) 
    client.set_timeout(30.0)
    

    world = client.get_world() 

    #Check if Town 4 is already loaded, if not load it
    if world.get_map().name != "Carla/Maps/Town04":
        #client.set_timeout(15.0) #BC mostly takes longer than default 5 Seconds
        client.load_world("Town04")
        world = client.get_world() 
        #Setting Spectator Camera Position to the 4-Way Intersection
        
    spectator = world.get_spectator()
    spectator.set_transform(carla.Transform(carla.Location(x=267.260406, y=-182.479721, z=17.611162), carla.Rotation(pitch=-47.555298, yaw=133.529037, roll=-0.000673)))

    #spectator.set_transform(carla.Transform(carla.Location(x = 255, y=-173, z=40),
    #carla.Rotation(pitch=-88.918983, yaw=-89.502739, roll=-0.001539)))
        

    return client


def spawn_entities(client, transform_list, entity_type):
    world = client.get_world()
    bp_lib = world.get_blueprint_library() 
  
    entity_list = []
    for transform in transform_list:
        if entity_type == "C":
            entity_bp = random.choice(bp_lib.filter('vehicle'))
        elif entity_type == "P":
            entity_bp = random.choice(bp_lib.filter('pedestrian'))
        entity =  world.try_spawn_actor(entity_bp, transform)
        entity_list.append(entity) 

    return entity_list

def transform_coordinates(df):
    cars = df[df["EntityType"]=="C"][["xCoord","yCoord","zCoord", "rotationDeg"]]
    passenger = df[df["EntityType"]=="P"][["xCoord","yCoord","zCoord","rotationDeg"]]

    coord_cars, coord_pass = [],[]

    for x in range(len(cars)):
        coord_cars.append(list(cars.iloc[x]))

    for x in range(len(passenger)):
        coord_pass.append(list(passenger.iloc[x]))


    transform_cars = []
    
    for element in  coord_cars:

        Location = carla.Location(x = element[0], y = element[1], z = element[2])
        Rotation = carla.Rotation(0, element[3], 0)
        transform = carla.Transform(Location, Rotation)

        transform_cars.append(transform)


    transform_pass = []
    
    for element in  coord_pass:

        Location = carla.Location(x = element[0], y = element[1], z = element[2])
        Rotation = carla.Rotation(0, element[3], 0)
        transform = carla.Transform(Location, Rotation)

        transform_pass.append(transform)

    return transform_cars, transform_pass

def scale_coords(df):
    carla_urx, carla_ury = 256.63, -170.97
    carla_difx, carla_dify = 28.28, 28.28
    
    df[["xCoord", "yCoord"]] = df[["xCoord", "yCoord"]] / 400

    df["xCoord"] = df["xCoord"] * carla_difx + carla_urx
    df["yCoord"] = df["yCoord"] * carla_dify + carla_ury

    return df

def read_gui_input(path):
    df = pd.read_csv(path)   

    return df

def control_cars(vehicles):
    for car in vehicles[1:]:
        car.apply_control(carla.VehicleControl(throttle = 0.5))

    start_autopilot(vehicles[0])

    

def control_dummy_passenger(passengers):
    for passenger in passengers:
        passenger.apply_control(carla.WalkerControl(speed = 0.55))

def start_autopilot(vehicle):
    vehicle.set_autopilot(True)


def start_recording():
   path = "C:\\Users\\stefan.stingl\\CARLA_0.9.13\\WindowsNoEditor"
   s1 = "cd " + path + "\\PythonAPI\\examples" + " && "
   s1 = s1 + "python start_recording.py -f " + path + "\\recording01.log -n 0 -t 12" 
   print(s1)
   run(s1)

def replay_recording():
    client.stop_recorder()
    
    path = "C:\\Users\\Natalie\\Desktop\\Carla\\WindowsNoEditor\\recording01.log"
    print(client.replay_file(path)) # replay the session

    # show collisions
    print(client.show_recorder_collisions(path, "v", "a"))

In [82]:
client = setup_world()

In [83]:
df = read_gui_input("data\\demo_3Cars.txt")
df

,EntityType,xCoord,yCoord,zCoord,rotationDeg
0,C,363,16,0.3,180
1,C,26,-217,0.3,90
2,C,-98,-24,0.3,0
3,P,29,-92,0.3,270
4,P,-56,-99,0.3,0
5,P,-77,78,0.3,0


In [84]:
df = scale_coords(df)
df

,EntityType,xCoord,yCoord,zCoord,rotationDeg
0,C,282.2941,-169.8388,0.3,180
1,C,258.4682,-186.3119,0.3,90
2,C,249.7014,-172.6668,0.3,0
3,P,258.6803,-177.4744,0.3,270
4,P,252.6708,-177.9693,0.3,0
5,P,251.1861,-165.4554,0.3,0


In [85]:
vehicle_list, passenger_list = transform_coordinates(df)
print(vehicle_list[0])

Transform(Location(x=282.294098, y=-169.838806, z=0.300000), Rotation(pitch=0.000000, yaw=180.000000, roll=0.000000))


In [86]:
vehicles = spawn_entities(client, vehicle_list, "C")
vehicles

[None,

In [87]:
passen = spawn_entities(client, passenger_list, "P")
passen

 None]

In [88]:


control_cars(vehicles)
passen_list = []

for p in passen:
    if type(p) != type(None):
        passen_list.append(p)

passen = passen_list
control_dummy_passenger(passen)


AttributeError: 'NoneType' object has no attribute 'set_autopilot'

In [89]:
start_recording()


cd C:\Users\stefan.stingl\CARLA_0.9.13\WindowsNoEditor\PythonAPI\examples && python start_recording.py -f C:\Users\stefan.stingl\CARLA_0.9.13\WindowsNoEditor\recording01.log -n 0 -t 12
Command executed succesfully
b'found 372 spawn points.\r\nRecording on file: C:\\Users\\stefan.stingl\\CARLA_0.9.13\\WindowsNoEditor\\recording01.log\r\nspawned 0 vehicles, press Ctrl+C to exit.\r\n\r\ndestroying 0 actors\r\nStop recording\r\n\r\ndone.\r\n'


In [91]:

path = "C:\\Users\stefan.stingl\\CARLA_0.9.13\\WindowsNoEditor\\recording01.log"
print(client.replay_file(path, 0.0, 0, 394))

print(client.show_recorder_collisions(path, "a", "a"))
print(client.show_recorder_actors_blocked(path, 0, 0))

Replaying File: C:\Users\stefan.stingl\CARLA_0.9.13\WindowsNoEditor\recording01.log
Total time recorded: 12.3914
Replaying from 0 s - 12.3914 s (12.3914 s) at 1.0x

Version: 1
Map: Town04
Date: 10/06/22 14:18:28

    Time  Types     Id Actor 1                                 Id Actor 2                            

Frames: 502
Duration: 12.3914 seconds

Version: 1
Map: Town04
Date: 10/06/22 14:18:28

    Time     Id Actor                                 Duration
       0     24 spectator                                    0
       0    393 vehicle.nissan.micra                         0
       0    394 vehicle.nissan.micra                         0
       0    392 vehicle.harley-davidson.low_rider            0
       0    401 walker.pedestrian.0004                       0
       0    400 walker.pedestrian.0029                       0
       0    399 vehicle.volkswagen.t2_2021                   0
       0    396 walker.pedestrian.0009                       0
       0    397 walker.pedestr

: 

In [19]:
import os
import sys
os.environ["CARLA_ROOT"] = 'C:/CARLA/WindowsNoEditor'
os.environ["SCENARIO_RUNNER_ROOT"] = "C:/Users/jonas/OneDrive - bwedu/Studium_Master/2_HWS_2022-23/AI Testing Simulator/code/AI_Testing_Simulator/scripts/runnerTool"



sys.path.append('/path/to/whatever')


set PYTHONPATH=%CARLA_ROOT%\PythonAPI\carla\dist\carla-0.9.13.egg
set PYTHONPATH=%PYTHONPATH%;%CARLA_ROOT%\PythonAPI\carla\agents
set PYTHONPATH=%PYTHONPATH%;%CARLA_ROOT%\PythonAPI\carla

C:/CARLA/WindowsNoEditor
C:/CARLA/WindowsNoEditor
